In [1]:
### Загружаем библиотеки и данные
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from scipy.stats import norm

In [3]:
data = load_iris()
X, y, column_names = data['data'], data['target'], data['feature_names']
X = pd.DataFrame(X, columns = column_names)

In [4]:
### Разбиваем данные
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=44)
 

In [5]:
means = X_train.groupby(y_train).apply(np.mean)
stds = X_train.groupby(y_train).apply(np.std)

C:\Games\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [6]:
### Вычисляем априорную вероятность класса
probs = X_train.groupby(y_train).apply(lambda x: len(x)) / X_train.shape[0]
 

In [7]:
### Вычисляем вероятность для Теоремы Байеса для каждого элемента
y_pred = []
# каждый элемент в валидационной части данных
for elem in range(X_val.shape[0]):
   p = {}
 
   # для каждого возможного класса
   for cl in np.unique(y_train):
 
       # априорная вероятность взятого ранее класса
       p[cl] = probs.iloc[cl]
 
       # для каждого столбца в датасете
       for index, param in enumerate(X_val.iloc[elem]):
 
           # умножаем вероятность того, что данное значение столбца
           # будет принадлежать распределению для выбранного класса
           p[cl] *= norm.pdf(param, means.iloc[cl, index], stds.iloc[cl, index])
  
   y_pred.append(pd.Series(p).values.argmax())
 

In [14]:
### Посмотрим точность нашего предсказания несколькими методами
# ручной классификатор
from sklearn.metrics import accuracy_score
accuracy1 = accuracy_score(y_val, y_pred)

In [15]:
# классификатор из библиотеки sklearn
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
accuracy2 = accuracy_score(y_val, model.predict(X_val))

In [16]:
print(accuracy1)
print(accuracy2)

0.9210526315789473
0.9210526315789473
